# Final Project Script for Loading Data

### API for GBIF: https://pygbif.readthedocs.io/en/latest/index.html

## Get imports for pygbif library and image loading

In [1]:
from pygbif import species as species
from pygbif import occurrences as occurrences
from IPython.display import Image, display
import requests

## For our usecase, we are looking at plants around Knoxville, so we are going to use Knoxville's coordinates

In [2]:
latitude = 35.964668
longitude = -83.926453

#We need to make the geometry a polygon, so we're going to make a square out of these coords.

min_lat = latitude - 0.09
max_lat = latitude + 0.09
min_lon = longitude - 0.12
max_lon = longitude + 0.12

## Search for occurrences within the box

In [3]:
results = occurrences.search(
    kingdomKey=6,
    decimalLatitude=f"{min_lat}, {max_lat}",
    decimalLongitude=f"{min_lon},{max_lon}",
    facet='speciesKey',
    limit = 0,
)

## Generate results (DEPRECIATED)

In [4]:
for record in results['results']:
    count = 0
    species_name = record.get('species') or record.get('scientificName')
    print(f"Species: {species_name}")
    if 'media' in record:
        for media in record['media']:
            url = media.get('identifier')
            #try:
            #    response = requests.get(url)
            #    display(Image(data=response.content))
            #except:
                #print("Could not Print")
    #print(count)


In [5]:
species_keys = [int(f['name']) for f in results['facets'][0]['counts']] 

In [6]:
images = {}

for key in species_keys:
    img_result = occurrences.search(
        speciesKey = key,
        mediatype='StillImage',
        limit=300
    )
    urls = []
    for rec in img_result['results']:
        for m in rec.get('media', []):
            if m.get('identifier'):
                urls.append(m['identifier'])
    images[key] = urls

# Example: how many images for the first species?
first = species_keys[0]
print(f"SpeciesKey {first} has {len(images[first])} images.")
for url in images[first][:5]:
    display(Image(url=url))

SpeciesKey 7902463 has 602 images.


In [8]:
from transformers import CLIPProcessor, CLIPModel
import torch

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def get_clip_embedding(img_path):
    image = Image.open(img_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
    return outputs[0]  # shape: [512]

/home/jacob/CodeSpace/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-04 17:52:26.186521: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-04 17:52:26.377915: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746395546.453098    9435 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746395546.475982    9435 cuda_blas.cc:1418] Un

In [11]:
get_clip_embedding(url)

AttributeError: type object 'Image' has no attribute 'open'